In [13]:
import pandas as pd

PATH = "../executions/MNQM2022/20220318-20220325.csv"


def get_executions_df(file_path: str):
    df = pd.read_csv(file_path)
    df = df[df["realized_pnl"].notnull()]
    df["date"] = pd.to_datetime(df["time_completed"]).dt.date

    df_grouped = df.groupby("date").sum()[["realized_pnl", "px_side"]]

    df_grouped["pnl / side"] = df_grouped["realized_pnl"] / df_grouped["px_side"]
    df_grouped["profit"] = df[df["realized_pnl"] > 0].groupby("date").count()["realized_pnl"]
    df_grouped["loss"] = df[df["realized_pnl"] < 0].groupby("date").count()["realized_pnl"]
    df_grouped.loc["Total"] = df_grouped.sum(numeric_only=True)

    return df_grouped


get_executions_df(PATH)

,realized_pnl,px_side,pnl / side,profit,loss
date,,,,,
2022-03-18,2326.020001,320.211667,7.264008,14.0,5.0
2022-03-21,-1456.600001,-306.435834,4.753361,12.0,15.0
2022-03-22,943.200000,117.325000,8.039207,3.0,2.0
2022-03-23,-261.500000,-47.263333,5.532830,5.0,9.0
2022-03-24,717.280000,145.170000,4.940966,6.0,3.0
2022-03-25,-69.179999,-76.455000,0.904846,17.0,12.0
Total,2199.220001,152.552500,31.435218,57.0,46.0


In [14]:
def get_executions_ungroup(file_path: str):
    df = pd.read_csv(file_path)
    df = df[df["realized_pnl"].notnull()]
    df["date"] = pd.to_datetime(df["time_completed"]).dt.date

    cols = ["quantity", "realized_pnl", "px_side"]

    return (
        df,
        df.groupby("date").sum()[cols],
        df.groupby("date").count()[["realized_pnl"]]
        # Only a single column is needed for showing the total realization count
    )


df_all, df_sum, df_count = get_executions_ungroup(PATH)

,contract,time_completed,side,quantity,avg_price,realized_pnl,epoch_sec,profit,loss,wr,...,avg_pnl_loss,avg_pnl_rr,avg_pnl_ewr,total_px_profit,avg_px_profit,total_px_loss,avg_px_loss,avg_px_rr,avg_px_ewr,date
2,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-18 01:09:43.000,SLD,2,14026.50,155.720000,1.647584e+09,1,0,1.000000,...,NaN,NaN,NaN,38.930000,38.930000,0.000000,NaN,NaN,NaN,2022-03-18
5,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-18 02:03:43.000,SLD,1,14014.25,-41.306667,1.647587e+09,1,1,0.500000,...,-41.306667,3.769851,0.209650,0.000000,38.930000,-20.653333,-20.653333,1.884926,0.346629,2022-03-18
6,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-18 02:51:33.000,SLD,2,14037.50,10.386667,1.647590e+09,2,1,0.666667,...,-41.306667,2.010652,0.332154,41.526667,20.763333,0.000000,-20.653333,1.005326,0.498672,2022-03-18
8,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-18 05:09:56.000,SLD,3,14034.50,11.580000,1.647598e+09,3,1,0.750000,...,-41.306667,1.433882,0.410866,43.456667,14.485556,0.000000,-20.653333,0.701366,0.587763,2022-03-18
10,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-18 06:08:28.000,SLD,3,13990.00,-82.920000,1.647602e+09,3,2,0.600000,...,-62.113334,0.953562,0.511886,0.000000,14.485556,-34.473333,-17.236667,0.840392,0.543363,2022-03-18
13,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-18 06:43:04.000,SLD,3,14013.75,157.580001,1.647604e+09,4,2,0.666667,...,-62.113334,1.349415,0.425638,69.720000,17.430000,0.000000,-17.236667,1.011216,0.497212,2022-03-18
15,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-18 08:19:33.000,SLD,3,13999.50,89.580000,1.647610e+09,5,2,0.714286,...,-62.113334,1.367973,0.422302,84.650000,16.930000,0.000000,-17.236667,0.982208,0.504488,2022-03-18
17,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-18 08:33:03.000,SLD,1,14059.00,68.360000,1.647610e+09,6,2,0.750000,...,-62.113334,1.323405,0.430403,118.830000,19.805000,0.000000,-17.236667,1.149004,0.465332,2022-03-18
20,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-18 08:36:00.000,SLD,2,14055.00,109.220000,1.647611e+09,7,2,0.777778,...,-62.113334,1.385547,0.419191,146.135000,20.876429,0.000000,-17.236667,1.211164,0.452251,2022-03-18
23,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-18 08:51:25.000,SLD,3,14117.50,436.080000,1.647611e+09,8,2,0.800000,...,-62.113334,2.089943,0.323631,218.815000,27.351875,0.000000,-17.236667,1.586842,0.386572,2022-03-18


In [4]:
df_sum


,quantity,realized_pnl,px_side
date,,,
2022-03-18,57,2326.020001,320.211667
2022-03-21,90,-1456.600001,-306.435834
2022-03-22,20,943.200000,117.325000
2022-03-23,50,-261.500000,-47.263333
2022-03-24,23,717.280000,145.170000
2022-03-25,87,-69.179999,-76.455000


In [5]:
df_count


,realized_pnl
date,
2022-03-18,19
2022-03-21,27
2022-03-22,5
2022-03-23,14
2022-03-24,9
2022-03-25,29


In [6]:
def get_executions_ungroup(file_path: str):
    df = pd.read_csv(file_path)
    df = df[df["realized_pnl"].notnull()]
    df["date"] = pd.to_datetime(df["time_completed"]).dt.date

    return (
        df[df["realized_pnl"] > 0].groupby("date").sum()[["realized_pnl"]],
        df[df["realized_pnl"] < 0].groupby("date").sum()[["realized_pnl"]]
    )


df_pos, df_neg = get_executions_ungroup(PATH)

In [7]:
df_pos


,realized_pnl
date,
2022-03-18,2943.506668
2022-03-21,3122.900000
2022-03-22,1809.460000
2022-03-23,1888.120000
2022-03-24,1050.760000
2022-03-25,2184.020001


In [8]:
df_neg


,realized_pnl
date,
2022-03-18,-617.486667
2022-03-21,-4579.500001
2022-03-22,-866.260000
2022-03-23,-2149.620000
2022-03-24,-333.480000
2022-03-25,-2253.200000
